<a href="https://colab.research.google.com/github/suhaodatascichem/NIR-calibration---NN/blob/main/NIR%20calibration%20-%20NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# read bruker files - brukeropusreader
!pip install brukeropusreader

  Preparing metadata (setup.py) ... done
  Created wheel for brukeropusreader: filename=brukeropusreader-1.3.4-py3-none-any.whl size=5303 sha256=215b373e8a98f62ea75ff9fee8f76c2ec2b1d91e721577b24f1b354976abf6c6
  Stored in directory: /root/.cache/pip/wheels/33/b6/88/6f85a1072c5d56dc11f6f28858fc7049281172fa48a5100908
Successfully built brukeropusreader


In [ ]:
import os
import numpy as np
import pandas as pd
from brukeropusreader import read_file
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Step 1: Read and Combine OPUS Files
data_directory = "C:/NIR calibration - NN"

def load_all_spectra(data_directory):
    all_data = []

    for file_name in os.listdir(data_directory):
        if file_name.endswith(".opus"):  # Check for OPUS files
            file_path = os.path.join(data_directory, file_name)
            data = read_file(file_path)  # Read OPUS file
            df = pd.DataFrame(data)  # Convert to DataFrame
            all_data.append(df)

    combined_df = pd.concat(all_data, ignore_index=True)  # Merge all DataFrames
    return combined_df

# Load all OPUS files
df = load_all_spectra(data_directory)

# Step 2: Preprocess Data
target_column = "AB"  # Ensure this column exists in your data
X = df.drop(columns=[target_column])  # Independent variables
y = df[target_column]  # Target variable

# Normalize input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Step 3: Build Neural Network Model
def build_model(input_shape):
    model = keras.Sequential([
        layers.Dense(64, activation="relu", input_shape=(input_shape,)),
        layers.Dense(32, activation="relu"),
        layers.Dense(1)  # Output layer (regression)
    ])

    model.compile(optimizer="adam", loss="mse", metrics=["mae"])
    return model

# Initialize and train the model
model = build_model(X_train.shape[1])
history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_split=0.2, verbose=1)

# Step 4: Evaluate the model
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=1)
print(f"Test MAE: {test_mae}")

# Make predictions
predictions = model.predict(X_test)
print("Predictions:", predictions[:5])  # Show first 5 predictions


In [ ]:
# Step 1: Read and Combine OPUS Files
data_directory = "C:/NIR calibration - NN"